<a href="https://colab.research.google.com/github/viicguimaraes14/Assignment01---CILP-implementation/blob/main/Assignment01_CILP_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Elton Dione and Victória Guimarães

#Imports

In [ ]:
import math
import random
import numpy as np
import pandas as pd

# Neural Network Def

In [ ]:
random.seed(0)
def rand(a, b):
  return (b-a)*random.random() + a

def sigmoid(x, diff=False):
  if diff:
    return 1.0 - x**2
  else:
    return math.tanh(x)

class back:
  def __init__(self, ni, nh, no, pesos, w):
    self.ni = ni + 1 
    self.nh = nh 
    self.no = no

    self.ai = [1.0]*self.ni
    self.ah = [1.0]*self.nh
    self.ao = [1.0]*self.no

    self.wi = np.zeros((self.ni, self.nh))
    self.wo = np.zeros((self.nh, self.no))

    self.wi = pesos

    for j in range(self.nh):
      for k in range(self.no):
        self.wo[j][k] = rand(-0.2, 0.2)
    print(self.wo)

    self.ci = np.zeros((self.ni, self.nh))
    self.co = np.zeros((self.nh, self.no))

  def update(self, inputs):
    if len(inputs) != self.ni-1:
      raise ValueError('input error')

    for i in range(self.ni-1):
      self.ai[i] = inputs[i]

    for j in range(self.nh):
      sum = 0.0
      for i in range(self.ni):
          sum = sum + self.ai[i] * self.wi[i][j]
      self.ah[j] = sigmoid(sum)

    for k in range(self.no):
      sum = 0.0
      for j in range(self.nh):
          sum = sum + self.ah[j] * self.wo[j][k]
      self.ao[k] = sigmoid(sum)

    return self.ao[:]

  def backpropagation(self, targets, N, M):
    if len(targets) != self.no:
      raise ValueError('wrong number of target values')

    output_deltas = [0.0] * self.no
    for k in range(self.no):
      error = targets[k]-self.ao[k]
      output_deltas[k] = sigmoid(self.ao[k], diff=True) * error

    hidden_deltas = [0.0] * self.nh
    for j in range(self.nh):
      error = 0.0
      for k in range(self.no):
          error = error + output_deltas[k]*self.wo[j][k]
      hidden_deltas[j] = sigmoid(self.ah[j], diff=True) * error

    for j in range(self.nh):
      for k in range(self.no):
        change = output_deltas[k]*self.ah[j]
        self.wo[j][k] = self.wo[j][k] + N*change + M*self.co[j][k]
        self.co[j][k] = change

    for i in range(self.ni):
      for j in range(self.nh):
        change = hidden_deltas[j]*self.ai[i]
        self.wi[i][j] = self.wi[i][j] + N*change + M*self.ci[i][j]
        self.ci[i][j] = change

    error = 0.0
    for i in range(len(targets)):
      error = error + 0.5*(targets[i]-self.ao[i])**2

    return error

  def test(self, patterns):
    for p in patterns: print(p[0], '->', self.update(p[0]))

  def weights(self):
    print(' Input weights:')
    for i in range(self.ni): print(self.wi[i])

    print('\n Output weights:')
    for j in range(self.nh): print(self.wo[j])

  def train(self, patterns, iterations=2598, N=0.5, M=0.1):
    for i in range(iterations):

      error = 0.0
      for p in patterns:
        inputs = p[0]
        targets = p[1]
        self.update(inputs)
        error = error + self.backpropagation(targets, N, M)
      if i % 200 == 0:
        print('error %-.5f' % error)


# CILP

In [ ]:
def funcH(x,b):
  aux = math.exp(-b*x)
  return ((2/(1 + aux)) - 1)

# read clausules
def lerP():
  fim = "n"
  P = []
  while(fim == "n"):
    cabeca = input("")
    corpo = input("")
    r = [cabeca, corpo.split(',')]
    P.append(r)
    # print(P)
    fim = input("y/n : ")
  return P

# use to know a negation variable
def isNot(x, ver=False):
  
  if ver:
    if x!='':
      if x[0]=="-":
        return True
      else: return False
    else: return False

  else:
    if x[0]=="-":
      return x[1]
    else:
      return "-"+x

# get litearais
def literais(P):
  all = []
  for r in P:
    for s in r:
      for t in s:
        if (t not in all) and (t != '') and (isNot(t) not in all):
          if isNot(t,True):
            all.append(isNot(t))
          else:
            all.append(t)
  return all

def neg_pos(P):
  Neg = []
  Pos = []
  for k in P:
    n=0
    p=0
    for r in k[1]:
      if isNot(r,True):
        print("neg: "+r)
        n+=1
      else:
        if r!='':
          print("pos:  "+r)
          p+=1

    Neg.append(n)
    Pos.append(p)
    print()
  return Neg,Pos

# get values from data
def q_v_k(P):
  q = len(P)
  v = len(literais(P))
  K = []
  k = 0
  for r in P:
    print(r[1], len(r[1]))
    if r[1]!=['']:
      K.append(len(r[1]))
    else:
      K.append(0)

  return q,v,K

def u_heads(P):
  heads = [a[0] for a in P]
  return [heads.count(a) for a in heads]

# variable MAXp
def Maxr(k,u):
  if k>u: return k
  else: return u

def Maxp(k,u):
  max = 0
  for i in k:
    for j in u:
      if Maxr(i,j)>max:
        max = Maxr(i,j)
  return max

# variable Amin
def Amin(k,u):
  return ((Maxp(k,u)-1) / (Maxp(k,u)+1))

def W_pesos(k,u,b, amin):
  return (2/b)*(math.log(1+amin)-math.log(1-amin))/(Maxp(k,u)*(amin-1) + amin+1)

# hidden layer
def threshold_hidden(amin, K, W):
  return [(1+amin)*(k-1)*W/2 for k in K]

def empyt(x):
  if x=='':
    return 0
  else: return 1

def negacoes(c):
  n = c[1]
  for i in range(len(n)):
    if isNot(n[i], True):
      n[i]=isNot(n[i])
  return n
  
def get_data(file_path):
    file_content = pd.read_csv(file_path, sep="\n", header=None)[0].tolist() 
    data = []
    for s in file_content:
        data.append(eval(s))
    return data

In [ ]:
def pesos_clauses(P, w, amin):
  pesos = []
  for r in P:
    l = literais(P)
    l.append('Bias')
    dicio = dict.fromkeys(l, 0)
    try:
        while True:
            r[1].remove('')
    except ValueError:
        pass
    values = [-1 if isNot(a, True) else empyt(a) for a in r[1]]
    keys = negacoes(r)
    for i in range(len(keys)):
      dicio[keys[i]] = values[i]
    pesos.append(list(dicio.values()))
  
  for i in range(len(pesos)):
    for a in range(len(pesos[i])):
      # print(a)
      pesos[i][a]=pesos[i][a]*w
  pesos = np.transpose(pesos)
  print(pesos)
  q, v, K = q_v_k(P)
  pesos[-1] = threshold_hidden(amin, K, w)
  return pesos

In [ ]:
P = [['A', ['B', '-C','-D']], ['A', ['E', 'F']], ['B', ['']], ['C', ['-B']]]
pesos = pesos_clauses(P,4.5,0.7)

[[ 0.   0.   0.   0. ]
 [ 4.5  0.   0.  -4.5]
 [-4.5  0.   0.   0. ]
 [-4.5  0.   0.   0. ]
 [ 0.   4.5  0.   0. ]
 [ 0.   4.5  0.   0. ]
 [ 0.   0.   0.   0. ]]
['B', 'C', 'D'] 3
['E', 'F'] 2
[] 0
['B'] 1



# Training and Test process

## 3. training dataset should be passed in another file

--> Link to download the file and upload in this colab: https://docs.google.com/document/d/1raV4fyRGGgOW4ksvj1tXXpwK1ndkS5zYhVoOxrORqA8/edit?usp=share_link


In [ ]:
file_path = '/content/dataset_treino.txt'
training_dataset = get_data(file_path)
training_dataset

[[[0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0]],
 [[0, 1, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0]],
 [[0, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0]],
 [[1, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0]],
 [[1, 1, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0]],
 [[1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0]]]

## Training

In [ ]:
n = back(6, 4, 6, pesos, 4.5)
n.train(training_dataset)

[[ 0.0532355  -0.16661318  0.09022174  0.19472859 -0.03927327  0.071406  ]
 [-0.07352915 -0.11459014  0.08692966 -0.19905697  0.12909256  0.01133839]
 [-0.16088626 -0.15243844  0.05970617  0.14946153 -0.0880069   0.19140607]
 [-0.15992772  0.14157524 -0.04132153 -0.16746183 -0.09011446 -0.01880873]]
error 3.78048
error 0.00315
error 0.00149
error 0.00458
error 0.00315
error 0.00213
error 0.00133
error 0.00132
error 0.00087
error 0.00101
error 0.00106
error 0.00100
error 0.00086


## 4 test data should be given in a separated file

link: https://docs.google.com/document/d/1DIqV4J_416xpa3A_bAnkKHJdKF95Rch7OOtWQc_9VQ8/edit?usp=share_link 

In [ ]:
file_path = '/content/dataset_teste.txt'
test_dataset = get_data(file_path)
test_dataset

[[[0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0]],
 [[0, 1, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0]],
 [[0, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0]],
 [[1, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0]],
 [[1, 1, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0]],
 [[1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0]]]

##testing

In [ ]:
n.test(test_dataset)

[0, 0, 0, 0, 0, 0] -> [-0.004668951911664774, 0.9967762407571013, 0.9951895537657517, 5.1612177730890035e-06, -4.089327491689101e-07, -8.416203326098778e-07]
[0, 1, 1, 0, 0, 0] -> [-0.004771381032090319, 0.9962387439637894, 0.0032087134781172786, -1.0058143177173862e-05, 7.96925127234334e-07, 1.640143500480583e-06]
[0, 1, 0, 0, 0, 0] -> [0.9945977872534941, 0.9986789217961286, -0.002620036410628517, -9.000733977650116e-06, 7.131446564186693e-07, 1.467715765495112e-06]
[1, 1, 0, 0, 0, 0] -> [0.994681533947806, 0.9986532760533409, -0.004214886541224106, -8.507900491658567e-06, 6.740965557336877e-07, 1.387351266548616e-06]
[1, 1, 1, 0, 0, 0] -> [-0.002284684409572078, 0.9959904747417787, -0.0019751481456147667, -8.615076279011586e-06, 6.82588291097318e-07, 1.4048280206978189e-06]
[1, 0, 0, 0, 0, 0] -> [-0.0036552895916747746, 0.9967696395714449, 0.995188852965871, 5.215812422531447e-06, -4.1325838334127047e-07, -8.505228763133986e-07]
